<img src="./images/DLI_Header.png" style="width: 400px;">



# 3.0 멀티 노드 분산 학습 전략

이번 노트북에서는 멀티 노드에서 [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) GPT 사전 훈련을 실행하는 방법에 대해 알아보겠습니다.


## 목표

이번 노트북의 목표는 다음과 같습니다:
* Megatron-LM 스크립트의 간단한 멀티 노드 학습 실행
* 데이터, 텐서 및 파이프라인 병렬 분포를 사용하여 하이브리드 다중 노드 실행


**[3.1 Megatron-LM GPT 사전 훈련의 멀티 노드 트레이닝 실행](#1.1-The-hardware-overview)<br>**
**[3.2 데이터 병렬화로 멀티-노드 실행](#1.1-The-hardware-overview)<br>**
**[3.3 인터/인트라 노드 커뮤니케이션](#1.1-The-hardware-overview)<br>**
**[3.4 프로파일링](#1.1-Profiling)<br>**
**[3.5 연습: 하이브리드 분산 학습 전략](#1.1-The-hardware-overview)<br>**
**[3.6 GPU 별 배치 사이즈 증가하기](#1.1-The-hardware-overview)<br>**

### 이전 실행/보류 중인 작업 취소하기

다음으로 이동하기 전에 SLURM 대기열에서 아직 실행 중이거나 대기 중인 작업이 없는지 확인하십시오. 다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다:

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하여 `scancel`  명령어를 사용하여 모든 사용자의 작업을 취소합니다.

In [2]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 3.1 Megatron-LM GPT 사전 학습의 다중 노드 학습 실행

이전 노트북에서는 노드 1개를 할당한 후 대화형 세션에서 작업을 제출했습니다. 

멀티-노드 작업의 경우 SLURM 스케줄러를 활용해야 합니다. 기본적으로 Megatron-LM을 사용한 멀티 노드 훈련은 [PyTorch distributed launcher](https://pytorch.org/docs/stable/distributed.html)의 [NVIDIA Collective Communications Library - NCCL](https://developer.nvidia.com/nccl) 분산 백엔드를 사용합니다.

2-노드 실행의 경우`SBATCH`스크립트를 사용합니다. 파이썬 실행 명령과 해당 인자들는 단일 노드에서 수행된 방법과 유사하지만 리소스 할당을 위해 추가적인`SBATCH`인자가 필요합니다. 

먼저 데이터 병렬 만을 사용하여 2개 노드에 대해  Megatron-LM GPT 사전 훈련을 실행하는 것으로 시작합니다. 다시 말하면, 모델은 할당된 4개의 GPU에 복사되어 각각 다른 데이터 배치를 처리합니다.

# 3.2 데이터 병렬화를 통한 멀티 노드 실행

이전의 2-GPU 데이터 병렬 실행에서, 각 GPU에 의해 처리되는 배치 크기는`--global-batch-size`에서 설정된 글로벌 배치 크기 4에 해당하는 `--micro-batch-size`에서 설정된 2 였습니다. 

4개의 GPU를 사용할 때 GPU당 마이크로 배치 크기를 2로 유지하고 글로벌 배치 크기를 8로 설정할 수 있습니다(MICRO_BATCH_SIZE $ \times $ 4).

리소스를 할당하고 실행하기 전에 스크립트를 살펴보겠습니다. 

리소스 할당에 대한 `SBATCH` 인수를 확인하십시오.

In [3]:
# have a look at Megaton-LM GPT pretraining execution on 2 nodes
! cat /dli/code/pretrain_gpt_2Node4GPU.sh

#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=2
GLOBAL_BATCH_SIZE=8    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS

이제 sbatch [script pretrain_gpt_2Node4GPU.sh](./code/pretrain_gpt_2Node4GPU.sh)를 실행합니다.  `squeue` 명령어를 사용하여 SLURM 대기열을 확인합니다.


In [4]:
# submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU.sh

# Check the SLURM queue
!squeue

Submitted batch job 9
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 9  slurmpar dli_2nod    admin PD       0:00      2 (None)


마스터 랩 노드에서`nvidia-smi` 명령어를 실행하여 GPU 활용량을 확인할 수 있습니다. 몇 초 후 노드가 할당되고 스크립트가 실행을 시작할 때 아래와 같이 GPU 0,1,2,3이 사용되는 것을 볼 수 있습니다. 수업에서 처음으로 Megatron-LM을 실행하는 경우 코드를 컴파일하는 데 약 6분이 소요됩니다. 그 전까지는 GPU 활동을 볼 수 없습니다.


<img src="images/2N_4gpus_utilization.png" width="650"/>

In [5]:
# Check GPU utilization on the master node
! sleep 10
!nvidia-smi

Fri Mar 28 02:19:50 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   34C    P0    60W / 300W |   2154MiB / 16160MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

Megatron GPT-3 사전 훈련의 성능을 이해하기 위해 실행 중에 생성된 [로그](./megatron/logs/log_2Nodes4GPUS.txt)를 확인해 봅니다.

먼저 실행 내역의 전역 크기를 확인해 보겠습니다. 아래 부분을 확인해 보세요:
```
using world size: 4, data-parallel-size: 4, tensor-model-parallel size: 1, pipeline-model-parallel size: 1
```

In [6]:
! grep "using world size:" /dli/megatron/logs/log_2Nodes4GPUS.txt

using world size: 4, data-parallel-size: 4, tensor-model-parallel size: 1, pipeline-model-parallel size: 1 


이제 4개의 GPU에 대한 GPT 사전 훈련의 성능을 확인해 보겠습니다.

In [7]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS.txt

[Rank 0] (after 10 iterations) memory (MB) | allocated: 1907.2294921875 | max allocated: 10582.0205078125 | reserved: 10922.0 | max reserved: 10922.0
 iteration       10/     100 | consumed samples:           80 | elapsed time per iteration (ms): 808.1 | learning rate: 6.000E-05 | global batch size:     8 | lm loss: 1.048027E+01 | loss scale: 1.0 | grad norm: 4.595 | number of skipped iterations:   0 | number of nan iterations:   0 |


추출 로그에서 4개의 GPU (노드당 2개의 GPU)를 사용하는 동안 훈련 성능을 확인할 수 있습니다. 

```
iteration      100/     100 | consumed samples:          800 | elapsed time per iteration (ms): 537.3 | learning rate: 5.822E-05 | global batch size:     8 | lm loss: 7.448950E+00 | loss scale: 1.0 | grad norm: 1.303 | number of skipped iterations:   0 | number of nan iterations:   0 |
```

이 결과를 1개 노드 실행 결과와 비교해보고 강사와 함께 논의해 봅니다.

---
# 3.3 노드 내/노드 간 통신

이전 실행에서 아래와 같이 훈련 중에 NCCL 디버그 로그 트레이스를 출력하기 위해 NCCL 변수 `NCCL_DEBUG=INFO`를 추가한 것을 알 수 있습니다. 

![title](images/nodes_communication.png)


이를 통해 훈련 중에 GPU와 노드 간에 사용되는 네트워킹 유형을 확인할 수 있습니다. 

GPU-GPU 간 직접 통신은 `P2P/IPC` 로 다시 게시되는 반면 노드 내 통신은 `NET/Socket/0`을 통한 것으로 보고됩니다.
구성 환경에서 GPU 간 직접 통신은 GPU0<->GPU1 and GPU2<->GPU3 처럼 노드들 간에 사용되어야 합니다. 

이전 실행에서 NCCL이 기록한 [로그](./megatron/logs/log_2Nodes4GPUS.txt) 를 확인해 봅니다.

In [8]:
! grep Channel /dli/megatron/logs/log_2Nodes4GPUS.txt | grep slurmnode1

slurmnode1:1519:1686 [0] NCCL INFO Channel 00/02 :    0   1   2   3
slurmnode1:1519:1686 [0] NCCL INFO Channel 01/02 :    0   1   2   3
slurmnode1:1519:1686 [0] NCCL INFO Channel 00 : 3[1e0] -> 0[1b0] [receive] via NET/Socket/0
slurmnode1:1519:1686 [0] NCCL INFO Channel 01 : 3[1e0] -> 0[1b0] [receive] via NET/Socket/0
slurmnode1:1519:1686 [0] NCCL INFO Channel 00 : 0[1b0] -> 1[1c0] via direct shared memory
slurmnode1:1519:1686 [0] NCCL INFO Channel 01 : 0[1b0] -> 1[1c0] via direct shared memory
slurmnode1:1520:1687 [1] NCCL INFO Channel 00 : 1[1c0] -> 2[1d0] [send] via NET/Socket/0
slurmnode1:1520:1687 [1] NCCL INFO Channel 01 : 1[1c0] -> 2[1d0] [send] via NET/Socket/0
slurmnode1:1520:1687 [1] NCCL INFO Channel 00 : 1[1c0] -> 0[1b0] via direct shared memory
slurmnode1:1520:1687 [1] NCCL INFO Channel 01 : 1[1c0] -> 0[1b0] via direct shared memory
slurmnode1:1519:1686 [0] NCCL INFO Channel 00 : 2[1d0] -> 0[1b0] [receive] via NET/Socket/0
slurmnode1:1519:1686 [0] NCCL INFO Channel 01 : 2[

---
# 3.4 학습 모니터링 및 프로파일링

지금까지 Megatron-LM으로 학습 실행을 모니터링하는 작업은 텍스트 로그 파일을 통해 수행되었습니다. 그러나 하이퍼파라미터와 훈련/평가 지표의 Tensorboard 시각화를 통해 훈련 모니터링도 가능합니다. 또한 시각화는 훈련 중에 모델을 디버깅하고 최적화하는 데 도움이 될 수 있습니다.

## 3.4.1 Tensorboard에서 학습 메트릭 시각화

<img src="images/tensorboard1.png" width="950"/>

이전  Megatron-LM 실행에서 텐서보드 이벤트를 기록하기 위한 인수를 설정했습니다. 이전의 모든 실험의 그래프는 `megatron/tensorboard/` 폴더에서 확인할 수 있습니다.

다음 셀을 실행하여 브라우저 용 텐서보드 링크를 만듭니다. 그런 다음 링크를 클릭하면 지정된`Tensorboard` 디렉토리에 저장된 실험 메트릭 그래프를 볼 수 있습니다.

In [9]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

## 3.4.2 Tensorboard Pytorch 프로파일러

훈련 및 추론 프로세스 동안 병목 현상을 조사하는 데 몇 가지 기존 프로파일링 도구를 사용할 수 있습니다. 이를 통해 가장 비용이 많이 드는 연산, GPU 부족 또는 불필요한 연산과 같은 이슈 사항들을 식별할 수 있습니다. 

이번 과정에서는 Pytorch 실행 중 성능 메트릭을 수집하는 도구인 [Pytorch Profiler](https://pytorch.org/docs/stable/profiler.html) 를 사용합니다.  [Tensorboard-plugin](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html) 과 PyTorch Profiler를 함께 사용하면 하드웨어 가속기(TensorCore)를 사용하는지 여부에 관계없이 각 GPU에서 실행되는 몇 가지 연산 척도로 각 GPU 프로세스를 시각화할 수 있습니다. 또한 프로세스를 개선하기 위한 권장 사항도 제공합니다.

우리는 GPU_0의 실행만 추적하고, 2개의 훈련 단계 동안 연산을 모니터링하겠습니다. 우리는 `profile-execution`을 *True*로 설정하고`profile-name` 을 *baseline*로 설정하여 프로파일링을 명시했습니다.

이전 실행의 경우 프로파일링은 텐서보드 링크의  `pytorch_profiler`탭 에서 사용할 수 있습니다.

<img src="images/profiling1.png" width="900"/>

텐서보드 페이지를 이미 닫은 경우 다음 셀을 실행하여 링크를 다시 생성할 수 있습니다. 링크를 클릭하여 텐서보드를 연 다음  `PYTORCH_PROFILER` 탭으로 이동합니다.

In [10]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

프로파일러 페이지는 추적의 오버뷰를 전체적으로 보여줍니다. 


### 프로파일링 개요
홈페이지에는 여러 패널이 표시됩니다:
- GPU Summary: GPU 구성 및 활용도를 보여줍니다. 이전 실행에서는 혼합 정밀도 학습을 활성화하지 않았기 때문에 TensorCore가 사용되지 않습니다 (다음 노트북 참고). 

- Step Time Breakdown: 각 작업에 소요된 실행 시간을 보여줍니다. 이전 실행에서 스텝 시간의 48.5%는 통신에, 41.7%는 GPU의 커널 실행 시간에 활용되었습니다. 

- Performance Recommendation: 프로세스 개선 방법에 대한 권장 사항을 제공합니다. Gradient Accumulation (그래디언트 축적) 또는 배치 크기를 늘리는 등 이전 실행에 대한 몇 가지 권장 사항을 제공합니다. 또한 프로파일러는 커널 속도를 높이기 위해 자동 혼합 정밀도를 사용하도록 설정할 것을 제안합니다. 다음 섹션에서는 먼저 배치 크기를 늘리는 것으로 시작하여 프로세스에서의 영향(속도 향상 및 메모리 소비 측면)을 살펴보겠습니다.

### The Memory View
이제 실행 시 메모리 할당 및 할당 해제를 보여 주는 Memory View를 살펴보겠습니다. 메모리 곡선을 확대하면 관련 메모리 이벤트를 볼 수 있습니다.

<img src="images/profiling4.png" width="750"/>

메모리 할당과 할당 해제는 각각 순방향 및 역방향 패스에 해당합니다. 이 프로세스는 2개의 훈련 단계를 추적하면서 두 번 추적됩니다. 또한 훈련 단계에서 최대 10GB까지 GPU 장치 메모리가 사용됨을 알 수 있습니다. 모델이나 배치 크기를 늘려서 메모리 사용량을 늘릴 수 있습니다.

좋습니다!
다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [11]:
# Clean the checkpoints
! rm /dli/megatron/checkpoints/* -r 

---
# 3.5 GPU당 배치 크기 증가하기

**특별 경고:** 이번 섹션에서는 OOM(Out Of Memory) 문제가 예상됩니다! 걱정 안하셔도 됩니다. 다음 노트북에서 이 문제를 해결하는 몇 가지 방법을 살펴보겠습니다.


현재 GPT 모델의 크기는 GPU 메모리에 적합하므로 모델 분산 (텐서 또는 파이프라인 병렬화)가 필요하지 않습니다. \
데이터 병렬화만 사용하면 GPU 메모리에 맞는 최대 배치 크기에 도달할 때까지 GPU당 처리되는 배치 크기를 늘림으로써 훈련 처리량 (초당 처리되는 시퀀스 수)을 향상시킬 수 있습니다.\
각 GPU에서 처리되는 데이터를 두 배로 늘려 확인해 봅시다. 데이터 병렬화만 있는 4개의 GPU (DP_SIZE $= 4$) 를 사용할 경우 GPU 당 마이크로 배치 크기를 2에서 4로 늘림으로써 글로벌 배치 크기는 MICRO_BATCH_SIZE $\times$ TP_SIZE $= 16$ 이어야 합니다.

다음 셀을 실행하여 이번 시나리오에 대한 훈련 스크립트를 준비해 보겠습니다.

In [12]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4      # <--- CHANGED HERE
GLOBAL_BATCH_SIZE=16    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_DP_4_MBS_4"


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name DP_4_MBS_4 \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh


이제 이전 sbatch 스크립트 [pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh](./code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh) 를 제출합니다. 
 squeue 명령어을 사용하여 SLURM 대기열을 확인합니다.

In [13]:
# submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh

# Check the SLURM queue
!squeue

Submitted batch job 10
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                10  slurmpar dli_2nod    admin  R       0:01      2 slurmnode[1-2]


이제 생성된 Megatron GPT-3 사전 훈련 성능을 확인하기 위해 실행 중에 생성된 [로그](./megatron/logs/log_2Nodes4GPUS_hybrid_solution.txt) 파일을 확인해 보겠습니다.

In [14]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS_DP_4_MBS_4.txt

반복당 경과 시간이 표시되지 않습니다. **방금 무슨 일이 일어났을까요??!** 


우리는 GPU 메모리를 포화시키고 있습니까? 아래 셀을 실행하여 로그에서 RuntimeError 를 검색해 봅니다. 다음 메시지를 주목합니다.
```
RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 15.78 GiB total capacity; 13.90 GiB already allocated; 302.00 MiB free; 14.43 GiB reserved in total by PyTorch)
RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 1; 15.78 GiB total capacity; 13.90 GiB already allocated; 302.00 MiB free; 14.43 GiB reserved in total by PyTorch)
```

In [15]:
! grep "RuntimeError" /dli/megatron/logs/log_2Nodes4GPUS_DP_4_MBS_4.txt

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 15.78 GiB total capacity; 13.87 GiB already allocated; 398.00 MiB free; 14.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
RuntimeErrorRuntimeError: : CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 15.78 GiB total capacity; 13.87 GiB already allocated; 398.00 MiB free; 14.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONFCUDA out of memory. Tried to allocate 512.00 MiB (GPU 1; 15.78 GiB total capacity; 13.87 GiB already allocated; 398.00 MiB free; 14.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Mem

실제로 GPU 메모리가 지정된 인수를 사용하여 이 트랜스포머 모델 크기를 훈련할 수 없다는 것을 의미하는 `CUDA out of memory` 오류를 보게 되었습니다.

GPU당 처리되는 데이터의 양을 두 배로 늘리는 것은 16G 메모리에게는 소화하기 어렵습니다.  

다음 실습에서는 모델의 메모리 공간을 줄임으로써 이 문제를 해결하는 방법에 초점을 맞출 것입니다.

좋습니다. 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [16]:
# Clean the checkpoints and logs directory
! rm -rf /dli/megatron/checkpoints/* 

---
# 3.6 연습 : 하이브리드 분산 훈련 전략


다음 셀에서 "FIXME"를 수정하여 텐서 및 파이프라인 병렬로 2개의 노드(4개의 GPU)에서 새로운 Megatron-LM GPT 사전 훈련 실행을 구성해 보겠습니다. 

텐서와 파이프라인 병렬을 사용하기 위해, 텐서 병렬화를 위한 2개의 GPU와 파이프라인 병렬화를 위한 2개의 GPU를 사용하여 2차원으로 분산 모델을 유지할 수 있습니다. 따라서 리소스는 더 이상 데이터 병렬로 남아 있지 않습니다. 이 경우 `GLOBAL_BATCH_SIZE`는 `MICRO_BATCH_SIZE`와 동일한 크기로 다운그레이드해야 합니다.

막히면 [솔루션](solutions/ex3.4.ipynb)에서 힌트를 확인하십시오.

In [23]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=2
PP_SIZE=2

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4
GLOBAL_BATCH_SIZE=4

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_hybrid"


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name TP_PP \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh


이제 이전 sbatch 스크립트 [pretrain_gpt_2Node4GPU_hybrid.sh](/dli/code/pretrain_gpt_2Node4GPU_hybrid.sh) 를 실행하여 하이브리드 멀티 노드 실행을 진행합니다. `squeue` 명령어를 사용하여 SLURM 대기열을 확인합니다.

In [24]:
# Submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh

# Check the SLURM queue
!squeue

Submitted batch job 12
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                12  slurmpar dli_2nod    admin  R       0:01      2 slurmnode[1-2]


Megatron GPT-3 사전 훈련의 성능을 이해하기 위해, 우리는 실행 중에 생성된 로그를 확인할 수 있습니다.

[로그](/dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt)를 확인하면 하이브리드 실행에 대한 전역 크기를 확인할 수 있습니다. 아래 메세지를 확인해주세요.

```
using world size: 4, data-parallel-size: 1, tensor-model-parallel size: 2, pipeline-model-parallel size: 2
```

In [27]:
! grep "using world size:" /dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt

using world size: 4, data-parallel-size: 1, tensor-model-parallel size: 2, pipeline-model-parallel size: 2 


훈련 성능과 GPU0 메모리 할당 및 할당 해제 내용을 살펴보겠습니다. 다음과 유사한 그래프가 표시될 것 입니다. 
<img src="images/profiling_hybrid.png" width="950"/>

포워드 패스와 백워드 패스 사이의 일정한 시간 스텝은 얼마입니까? 강사님과 함께 논의해 봅시다.

In [28]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt

[Rank 1] (after 10 iterations) memory (MB) | allocated: 639.3779296875 | max allocated: 5250.76513671875 | reserved: 5478.0 | max reserved: 5478.0
[Rank 0] (after 10 iterations) memory (MB) | allocated: 640.3779296875 | max allocated: 5252.76513671875 | reserved: 5490.0 | max reserved: 5490.0
[Rank 2] (after 10 iterations) memory (MB) | allocated: 650.646484375 | max allocated: 5344.8046875 | reserved: 5662.0 | max reserved: 5662.0
 iteration       10/     100 | consumed samples:           40 | elapsed time per iteration (ms): 887.0 | learning rate: 6.000E-05 | global batch size:     4 | lm loss: 1.049753E+01 | loss scale: 1.0 | grad norm: 5.716 | number of skipped iterations:   0 | number of nan iterations:   0 |
[Rank 3] (after 10 iterations) memory (MB) | allocated: 651.771484375 | max allocated: 5345.3046875 | reserved: 5918.0 | max reserved: 5918.0
 iteration       20/     100 | consumed samples:           80 | elapsed time per iteration (ms): 525.3 | learning rate: 5.997E-05 | gl

좋습니다. 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [29]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/* 

---
<h2 style="color:green;">축하합니다!</h2>

다음으로 넘어가기 전에 대기열에서 실행 중이거나 대기 중인 작업이 없는지 확인합니다.

In [30]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하여 `scancel` 명령을 사용하여 모든 작업을 취소합니다.

In [31]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


다음으로, 혼합 정밀도, 그래디언트 누적 및 활성화 체크포인팅과 같은 기술을 사용하여 GPT 사전 훈련을 최적화하는 방법에 대해 알아보겠습니다. [004_GPT_LM_pretrainings_optimizations.ipynb](04_GPT_LM_pretrainings_optimizations.ipynb)로 이동합니다.